In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from scipy import stats
from sklearn.model_selection import train_test_split

In [76]:
data =  pd.read_excel("Motor Trend Car Road Tests.xlsx")
data.head()

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


1.1 Realiza una regresión tomando 'mpg' como salida y eliminando la columna 'model'. Considera todos los demás factores como numéricos/ordinales.

Calcula el R2 e interpreta los signos de los betas.
Realiza un train-test-split donde se use el 40% de los datos para entrenar. Calcula el R2 de entrenamiento y de prueba.
Añade regularización L2 con un hiperparámetro lambda decidido por ti. Cambia este valor y compara con varios distintos los R2 de entrenamiento y de prueba.


In [77]:
df = data.dropna().drop_duplicates()

x = df.drop(["model", "mpg"], axis=1)
y = df[["mpg"]]

In [78]:
lr = LinearRegression()
lr.fit(x, y)
lr.intercept_

array([12.30337416])

In [79]:
lr.coef_

array([[-0.11144048,  0.01333524, -0.02148212,  0.78711097, -3.71530393,
         0.82104075,  0.31776281,  2.52022689,  0.65541302, -0.19941925]])

In [80]:
y_pred = lr.predict(x)
r2_score(y, y_pred)

0.8690157644777647

Con las betas que tenemos podemos observar que hay ciertas variables que no tienen mucha relación con el mpg, como lo pueden ser los cilindros, los caballos de fuerza, y el carb. Aunque, para ser honesto diría que algunas de esas variables si podrían tener sentido que tuviesen influencia en el las millas por galón, pues los clindros claro que tienen que ver, y no creo que se muevan de forma contraria. Sin embargo, tenemos un r2 bastante algo así que eso me hace dudar

## **train-test split**

In [81]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.4, random_state=137)

In [82]:
scaler = StandardScaler().fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [83]:
rd = Ridge(alpha=1)
rd.fit(x_train_scaled, y_train)

y_train_pred = rd.predict(x_train_scaled)
y_test_pred = rd.predict(x_test_scaled)

In [84]:
r2_rd_mpg_train = r2_score(y_train, y_train_pred)
r2_rd_mpg_test = r2_score(y_test, y_test_pred) 

In [85]:
for alpha in range(1, 11):  # valores de 1 a 10
    rd = Ridge(alpha=alpha)
    rd.fit(x_train_scaled, y_train)
    
    y_train_pred_rd = rd.predict(x_train_scaled)
    y_test_pred= rd.predict(x_test_scaled)
    
    r2_test = r2_score(y_test, y_test_pred)
    r2_train = r2_score(y_train, y_train_pred)
    
    print(f"Alpha: {alpha}")
    print(f"  Intercepto: {rd.intercept_}")
    print(f"  Coeficientes: {rd.coef_}")
    print(f"  R²_test: {r2_test}")
    print(f"  R²_train: {r2_train}")
    print("-" * 40)


Alpha: 1
  Intercepto: [17.43333333]
  Coeficientes: [[-1.80358779 -1.2857849   1.01328527  1.67274528 -0.7201921   1.26762625
   0.01863527  1.40125429 -1.20813325 -0.75672773]]
  R²_test: 0.5910911509384542
  R²_train: 0.9386746881405903
----------------------------------------
Alpha: 2
  Intercepto: [17.43333333]
  Coeficientes: [[-1.55710774 -1.04129282  0.27022389  1.30415131 -0.78349879  0.95871328
   0.16070631  1.20840106 -0.64674854 -0.65854085]]
  R²_test: 0.6660803311091934
  R²_train: 0.9386746881405903
----------------------------------------
Alpha: 3
  Intercepto: [17.43333333]
  Coeficientes: [[-1.41075875 -0.95972866 -0.03154439  1.13631442 -0.79865506  0.81845189
   0.24098542  1.09006687 -0.38665798 -0.61529874]]
  R²_test: 0.6993439461168738
  R²_train: 0.9386746881405903
----------------------------------------
Alpha: 4
  Intercepto: [17.43333333]
  Coeficientes: [[-1.31213813 -0.91893497 -0.18947303  1.03589255 -0.80190612  0.73513012
   0.29411726  1.00538039 -0.2

Teniendo estos resultados de las r2 y de las betas podemos ver que existe mucho sobreajuste entre r2_test y train, ya que a pesar de que en el alpha 10 se están acercando más, aún así tienen cierta distancia entre uno y otro. 
Además, me brinca un poco también el hecho de que haga varias betas negativas, y no coinciden con las mismas de LinearRegression. 

**1.2 Repite el ejercicio anterior usando 'qsec' como salida.**

In [86]:
x = df.drop(["model", "qsec"], axis=1)
y = df[["qsec"]]

In [87]:
lr = LinearRegression()
lr.fit(x, y)
lr.intercept_, lr.coef_

(array([17.77617693]),
 array([[ 0.06904768, -0.36267823, -0.00750084, -0.00156255, -0.1310636 ,
          1.49633224,  0.97003474, -0.90118641, -0.20128511, -0.27359751]]))

In [88]:
y_pred = lr.predict(x)
r2_score(y, y_pred)

0.8746925777093999

## train-test spli

In [89]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.4, random_state=137)

scaler = StandardScaler().fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

rd = Ridge(alpha=1)
rd.fit(x_train_scaled, y_train)

y_train_pred = rd.predict(x_train_scaled)
y_test_pred = rd.predict(x_test_scaled)

r2_rd_qsec_train = r2_score(y_train, y_train_pred)
r2_rd_qsec_test = r2_score(y_test, y_test_pred) 

In [90]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.4, random_state=137)

scaler = StandardScaler().fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

for alpha in range(1, 11):  # valores de 1 a 10
    rd = Ridge(alpha=alpha)
    rd.fit(x_train_scaled, y_train)
    
    y_train_pred_rd = rd.predict(x_train_scaled)
    y_test_pred_rd = rd.predict(x_test_scaled)
    
    r2_test = r2_score(y_test, y_test_pred_rd)
    r2_train = r2_score(y_train, y_train_pred_rd)
    
    print(f"Alpha: {alpha}")
    print(f" Intercepto: {rd.intercept_}")
    print(f" Coeficientes: {rd.coef_}")
    print(f" R²_test: {r2_test}")
    print(f" R²_train: {r2_train}")
    print("-" * 40)


Alpha: 1
 Intercepto: [17.37333333]
 Coeficientes: [[ 0.32243014 -0.423128   -0.09911909 -0.48174143 -0.47038265  0.50302442
   0.45486151 -0.00142143 -0.19343426 -0.27205226]]
 R²_test: 0.6535717344645453
 R²_train: 0.9671802657602482
----------------------------------------
Alpha: 2
 Intercepto: [17.37333333]
 Coeficientes: [[ 0.26542739 -0.41081973 -0.09181096 -0.43831053 -0.38500361  0.38696105
   0.43204172 -0.0613563  -0.23679189 -0.26759015]]
 R²_test: 0.6575165192963931
 R²_train: 0.9562802655200474
----------------------------------------
Alpha: 3
 Intercepto: [17.37333333]
 Coeficientes: [[ 0.23649211 -0.38707656 -0.09242538 -0.41333718 -0.33564151  0.31680413
   0.41175484 -0.09645126 -0.25048246 -0.26808909]]
 R²_test: 0.6546296940253327
 R²_train: 0.9460042375824835
----------------------------------------
Alpha: 4
 Intercepto: [17.37333333]
 Coeficientes: [[ 0.21764208 -0.3648474  -0.09452839 -0.39531986 -0.30214276  0.26819891
   0.39376069 -0.11802265 -0.25488685 -0.268

En este análisis y resultados podemos notar que existe algo inusual, puesto que en los otros modelos a medida de que se iba incrementando el alpha el r2 lo hacía también, pero aquí esta ocurriendo lo contrario. Además, también se incrementaron los betas negativos, por lo que menos tienen influencia en qsec. 
Diría que también existe un sobreajuste, ya que hay mucha diferencia entre ambas r2.

**2.1 Realiza una regresión tomando 'mpg' como salida y eliminando la columna 'model'. Crea columnas dummies para los factores 'cyl', 'gear' y 'carb'.**

Calcula el R2 e interpreta los signos de los betas.

In [91]:
df = pd.get_dummies(df, columns=["cyl", "gear", "carb"], drop_first=True)

In [92]:
x = df.drop(["model", "mpg"], axis=1)
y = df[["mpg"]]

In [93]:
lr = LinearRegression()
lr.fit(x, y)

lr.intercept_, lr.coef_

(array([23.87913244]),
 array([[ 0.03554632, -0.07050683,  1.18283018, -4.52977584,  0.36784482,
          1.93085054,  1.2121157 , -2.64869528, -0.33616298,  1.11435494,
          2.52839599, -0.97935432,  2.99963875,  1.09142288,  4.47756921,
          7.25041126]]))

In [94]:
y_pred = lr.predict(x)

r2_dum_mpg = r2_score(y, y_pred)

Ahora habiendo hecho el data frame dummies, esto nos puede ayudar a que podamos observar con claridad cuales son las variables que tienen mayor influencia en el mpg de salida, que por ejemplo en el caso del carb solo afecta al número 2.
Y en cuanto al r2 si nos da bastante alto para considerarlo un buen modelo. 

#### Realiza un train-test-split donde se use el 40% de los datos para entrenar. Calcula el R2 de entrenamiento y de prueba.

In [95]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.4, random_state=137)

scaler = StandardScaler().fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [96]:
lr = LinearRegression()
lr.fit(x_train_scaled, y_train)

y_train_pred = lr.predict(x_train_scaled)
y_test_pred = lr.predict(x_test_scaled)

r2_dum_mpg_train = r2_score(y_train, y_train_pred)
r2_dum_mpg_test = r2_score(y_test, y_test_pred)

r2_dum_mpg_train, r2_dum_mpg_test

(1.0, 0.11565160877658709)

2.2 Repite el ejercicio anterior usando 'qsec' como salida.

In [97]:
x = df.drop(["model", "qsec"], axis=1)
y = df[["qsec"]]

In [98]:
lr = LinearRegression()
lr.fit(x, y)

lr.intercept_, lr.coef_

(array([16.3491375]),
 array([[ 2.77413839e-02,  3.53115688e-03, -2.06618843e-03,
          1.07865814e-01,  8.10472140e-01,  2.65732208e-01,
         -1.69429351e+00, -1.10434255e+00, -2.96690068e+00,
          1.33228234e+00,  1.82317492e-01, -8.34412978e-01,
         -2.29304367e-01, -1.94703381e+00, -1.56524104e+00,
         -1.33231717e+00]]))

In [99]:
y_pred = lr.predict(x)

r2_dum_qsec = r2_score(y, y_pred)
r2_dum_qsec

0.9082689440167556

#### Separando en trian y test

In [100]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.4, random_state=137)

scaler = StandardScaler().fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [101]:
lr = LinearRegression()
lr.fit(x_train_scaled, y_train)

y_train_pred = lr.predict(x_train_scaled)
y_test_pred = lr.predict(x_test_scaled)

r2_dum_qsec_train = r2_score(y_train, y_train_pred)
r2_dum_qsec_test = r2_score(y_test, y_test_pred)

r2_dum_qsec_train, r2_dum_qsec_test

(1.0, -2.3927070726210173)

**3.1 Compara los R2 de los ejercicios 1.1 & 2.1.**

**3.2 Compara los R2 de los ejercicios 1.2 & 2.2.**

In [103]:
print(f"R2 Ridge train qsec:{r2_rd_qsec_train} vs R2 dummies train qsec: {r2_dum_qsec_train}") 
print(f"R2 Ridge test qsec:{r2_rd_qsec_test} vs R2 dummies test qsec: {r2_dum_qsec_test}") 

print("   ")

print(f"R2 Ridge train mpg:{r2_rd_mpg_train} vs R2 dummies train mpg: {r2_dum_mpg_train}") 
print(f"R2 Ridge test mpg:{r2_rd_mpg_test} vs R2 dummies test mpg: {r2_dum_mpg_test}") 

R2 Ridge train qsec:0.9671802657602482 vs R2 dummies train qsec: 1.0
R2 Ridge test qsec:0.6535717344645453 vs R2 dummies test qsec: -2.3927070726210173
   
R2 Ridge train mpg:0.9386746881405903 vs R2 dummies train mpg: 1.0
R2 Ridge test mpg:0.5910911509384542 vs R2 dummies test mpg: 0.11565160877658709


En conclusión como se puede observar no obtuve buenos resultados en los dommies de los 4 r2. Me brinca la tanta diferencia que exista entre los r2, por lo que 100% podemos ver que exite overfitting. Y al obtener r2 de 1 vemos que se aprendió demasiado los datos y cuando le muestras algo distinto no es capaz de predecir con ello.

En cuando a Ridge si me dio muy bien. Bueno, si existe un poco de diferencia pero al menos no tanta como en dummies.